In [ ]:
import pandas as pd
import re
from statistics import mode, mean
import math 
import numpy as np

#SBIC 
remove id = -6837958490067487319, selected 33 times of 'man', 693 times of 'woman'. <br>
Demographic: Gender, Race, Age, Politics <br>
Post, Group_Disagreement, Personal_Disagreement, Group Disagreement Rate, Person Disagreement Rate 

In [ ]:
#SBIC 
#remove id = -6837958490067487319, selected 33 times of 'man', 693 times of 'woman'.
#Demographic: Gender, Race, Age, Politics
#Post, Group_Disagreement, Personal_Disagreement, Group Disagreement Rate, Person Disagreement Rate

In [ ]:
SBIC_Train = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Original_Datasets/SBIC/SBIC.v2.trn.csv')
SBIC_Dev = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Original_Datasets/SBIC/SBIC.v2.dev.csv')
SBIC_Test = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Original_Datasets/SBIC/SBIC.v2.tst.csv')
frames = [SBIC_Train, SBIC_Dev, SBIC_Test]
SBIC_total = pd.concat(frames)

In [ ]:
SBIC_total_Demographics = SBIC_total[['WorkerId','annotatorGender','annotatorPolitics','annotatorRace','annotatorAge']]
SBIC_Demographics = SBIC_total_Demographics.groupby('WorkerId')['annotatorGender','annotatorPolitics','annotatorRace','annotatorAge'].apply(lambda x: x.mode().iloc[0]).reset_index()
SBIC_Demographics.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


(307, 5)

In [ ]:
SBIC_total = pd.merge(SBIC_total, SBIC_Demographics, on="WorkerId", how="left")
SBIC_Disagreement = SBIC_total[['post','WorkerId', 'annotatorGender', 'annotatorRace', 'annotatorAge','annotatorPolitics','offensiveYN']]
SBIC_Disagreement = SBIC_Disagreement[SBIC_Disagreement.WorkerId != -6837958490067487319]

In [ ]:
SBIC_Disagreement['post'] = SBIC_Disagreement.post.apply(lambda x: re.sub(r'RT', '', x))
SBIC_Disagreement['post'] = SBIC_Disagreement.post.apply(lambda x: ' '.join([word for word in x.split() if not word.startswith('@')]))
SBIC_Disagreement['post'] = SBIC_Disagreement.post.apply(lambda x: re.sub(r'http\S+', '', x))
# remove punctuation marks
punctuation = ',!"$%&()*+-/:;<=>?@[\\]^_`{|}~'
SBIC_Disagreement['post'] = SBIC_Disagreement['post'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
# convert text to lowercase
SBIC_Disagreement['post'] = SBIC_Disagreement['post'].str.lower()
SBIC_Disagreement = SBIC_Disagreement.rename(columns={"post": "text"})
SBIC_Disagreement = SBIC_Disagreement.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
SBIC_Disagreement = SBIC_Disagreement.dropna() 

In [ ]:
#major vote, group_disagreement, person_disagreement 
major_vote = SBIC_Disagreement.groupby(['text'], as_index = False)['offensiveYN'].agg(pd.Series.mode)
major_vote =  major_vote.rename(columns={"offensiveYN": "major_vote"})
annotators_count = SBIC_Disagreement.groupby(['text'], as_index = False).size()
annotators_count = annotators_count.rename(columns={"size": "annotators_count"})
text_label_count = SBIC_Disagreement.groupby(['text','offensiveYN'], as_index = False).size()
text_label_count = text_label_count.rename(columns={"size": "text_label_count"})
SBIC_Disagreement = pd.merge(SBIC_Disagreement, major_vote, on="text", how="left")
SBIC_Disagreement = pd.merge(SBIC_Disagreement, annotators_count, on="text", how="left")
SBIC_Disagreement = pd.merge(SBIC_Disagreement, text_label_count, on=["text",'offensiveYN'], how="left")

In [ ]:
SBIC_Disagreement = SBIC_Disagreement[SBIC_Disagreement.annotators_count == 3]
SBIC_Disagreement['person_disagreement_rate'] = 1 - SBIC_Disagreement['text_label_count']/SBIC_Disagreement['annotators_count']
disagreement_rate = SBIC_Disagreement.groupby(['text'], as_index = False)['person_disagreement_rate'].agg(pd.Series.mode)
disagreement_rate = disagreement_rate.rename(columns={"person_disagreement_rate": "disagreement_rate"})
SBIC_Disagreement = pd.merge(SBIC_Disagreement, disagreement_rate, on="text", how="left")
SBIC_Disagreement['group_disagreement'] = [1 if SBIC_Disagreement['text_label_count'].iloc[x] /SBIC_Disagreement['annotators_count'].iloc[x]!=1 else 0 for x in range(len(SBIC_Disagreement))] 
SBIC_Disagreement['person_disagreement'] = [1 if SBIC_Disagreement['person_disagreement_rate'].iloc[x] != SBIC_Disagreement['disagreement_rate'].iloc[x]!=1 else 0 for x in range(len(SBIC_Disagreement))] 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#normalize disagreement rate
num_annotator = 3
most_controversial_rate = 1 - math.ceil(num_annotator/SBIC_Disagreement['offensiveYN'].nunique())/num_annotator
SBIC_Disagreement['normalized_disagreement_rate'] = SBIC_Disagreement['disagreement_rate'] / most_controversial_rate
SBIC_Disagreement['normalized_person_disagreement_rate'] = SBIC_Disagreement['person_disagreement_rate'] / most_controversial_rate


In [ ]:
SBIC_Disagreement.to_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/SBIC_Disagreement.csv',index=False)

#**Dilemmas**

In [ ]:
#Dilemmsa
Train_dilemmas = pd.read_json('/content/drive/MyDrive/Morality   Legality/Original_Datasets/dilemmas/train.scruples-dilemmas.jsonl', lines=True)
Dev_dilemmas = pd.read_json('/content/drive/MyDrive/Morality   Legality/Original_Datasets/dilemmas/dev.scruples-dilemmas.jsonl', lines=True)
Test_dilemmas = pd.read_json('/content/drive/MyDrive/Morality   Legality/Original_Datasets/dilemmas/test.scruples-dilemmas.jsonl', lines=True)

In [ ]:
def read_dilemmas_json(data):
  action = data['actions'].to_list()
  action = pd.DataFrame(action)
  action = action.rename(columns ={0:'first', 1:'second'})
  first = action['first'].to_list()
  first =pd.DataFrame(first)
  first = first.rename(columns ={'id':'first-id', 'description':'first-description'})
  second = action['second'].to_list()
  second =pd.DataFrame(second)
  second = second.rename(columns ={'id':'second-id', 'description':'second-description'})
  Dilemmas = pd.concat([data,first,second],axis=1)
  Dilemmas['text'] = Dilemmas['first-description'] +'. '+ Dilemmas['second-description'] + '.'
  Dilemmas['controversial'] = list(map(int, Dilemmas['controversial']))
  Dilemmas['disagreement_rate'] = [Dilemmas['gold_annotations'][x][0]/5 if Dilemmas['gold_annotations'][x][0]< Dilemmas['gold_annotations'][x][1] else Dilemmas['gold_annotations'][x][1]/5 for x in range(len(Dilemmas))]
  #normalize disagreement rate
  num_annotator = 5
  most_controversial_rate = 1 - math.ceil(num_annotator/2)/num_annotator
  Dilemmas['normalized_disagreement_rate'] = Dilemmas['disagreement_rate'] / most_controversial_rate  
  output = Dilemmas[['text','gold_annotations','controversial','disagreement_rate','normalized_disagreement_rate']]
  return output

train_dilemmas = read_dilemmas_json(Train_dilemmas)
dev_dilemmas = read_dilemmas_json(Dev_dilemmas)
test_dilemmas = read_dilemmas_json(Test_dilemmas)

In [ ]:
train_dilemmas.to_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/train_dilemmas_disagreement.csv',index=False)
dev_dilemmas.to_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/dev_dilemmas_disagreement.csv',index=False)
test_dilemmas.to_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/test_dilemmas_disagreement.csv',index=False)

In [ ]:
test_dilemmas['normalized_disagreement_rate'].value_counts()

0.5    808
0.0    780
1.0    772
Name: normalized_disagreement_rate, dtype: int64

#**Social Chemistry 101**

In [ ]:
anticipated_agreement=pd.read_csv('/content/drive/MyDrive/Morality   Legality/Processed_Datasets/Processed_social101/anticipated_agreement.csv')
demographic = pd.read_csv('/content/drive/MyDrive/Morality   Legality/Original_Datasets/Social_Chemistry101/demographics_publish.csv')
worker_demographic = demographic[['worker-id','gender','age', 'race','marital', 'economic', 'school', 'income', 'children', 'household','us', 'state', 'area', 'time-in-us']]
worker_demographic = worker_demographic.rename(columns = {'worker-id':'breakdown-worker-id'})
anticipated_agreement = anticipated_agreement.rename(columns={'rot': 'text','size': 'text_label_count'})

In [ ]:
anticipated_agreement.columns

Index(['m', 'text', 'rot-id', 'breakdown-worker-id', 'rot-agree-coded',
       'Major_rot_agree', 'Group_has_disagreement', 'person_disagreement',
       'text_label_count'],
      dtype='object')

In [ ]:
annotators_count = anticipated_agreement.groupby(['text'], as_index = False).size()
annotators_count = annotators_count.rename(columns={"size": "annotators_count"})
anticipated_agreement = pd.merge(anticipated_agreement, annotators_count, on="text", how="left")
anticipated_agreement = anticipated_agreement[(anticipated_agreement.m == 5) & (anticipated_agreement.annotators_count == 5)]

In [ ]:
anticipated_agreement['person_disagreement_rate'] = 1 - anticipated_agreement['text_label_count'] / anticipated_agreement['annotators_count']
disagreement_rate = anticipated_agreement.groupby(['text'], as_index = False)['person_disagreement_rate'].agg(pd.Series.mode)
disagreement_rate = disagreement_rate.rename(columns={"person_disagreement_rate": "disagreement_rate"})
anticipated_agreement = pd.merge(anticipated_agreement, disagreement_rate, on="text", how="left")

In [ ]:
#normalize disagreement rate
anticipated_agreement['normalized_disagreement_rate'] = [anticipated_agreement['disagreement_rate'].iloc[x] / (1 - math.ceil(anticipated_agreement['annotators_count'].iloc[x]/3)/anticipated_agreement['annotators_count'].iloc[x]) for x in range(len(anticipated_agreement))]
anticipated_agreement['normalized_person_disagreement_rate'] = [anticipated_agreement['person_disagreement_rate'].iloc[x] / (1 - math.ceil(anticipated_agreement['annotators_count'].iloc[x]/3)/anticipated_agreement['annotators_count'].iloc[x]) for x in range(len(anticipated_agreement))]

In [ ]:
anticipated_agreement['disagreement_rate'].value_counts()

0.0    21765
0.2     7730
0.4     5825
0.6      650
Name: disagreement_rate, dtype: int64

In [ ]:
anticipated_agreement['normalized_disagreement_rate'].value_counts()

0.000000    21765
0.333333     7730
0.666667     5825
1.000000      650
Name: normalized_disagreement_rate, dtype: int64

In [ ]:
social_agreement_disagreement = anticipated_agreement.merge(worker_demographic, on=['breakdown-worker-id'], how='left')

In [ ]:
social_agreement_disagreement.to_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/social_agreement_disagreement.csv',index=False)

#DynaSent

In [ ]:
r1_train = pd.read_json('/content/drive/MyDrive/Morality   Legality/Original_Datasets/dynasent-v1.1/dynasent-v1.1-round01-yelp-train.jsonl', lines=True)
r1_dev = pd.read_json('/content/drive/MyDrive/Morality   Legality/Original_Datasets/dynasent-v1.1/dynasent-v1.1-round01-yelp-dev.jsonl', lines=True)
r1_test = pd.read_json('/content/drive/MyDrive/Morality   Legality/Original_Datasets/dynasent-v1.1/dynasent-v1.1-round01-yelp-test.jsonl', lines=True)

In [ ]:
def read_dynasent_json(data):
  label_distribution = data['label_distribution'].to_list()
  label_distribution=pd.DataFrame(label_distribution)
  label_distribution['num_positive']=label_distribution['positive'].apply(lambda x: len(x))
  label_distribution['num_negative']=label_distribution['negative'].apply(lambda x: len(x))
  label_distribution['num_neutral']=label_distribution['neutral'].apply(lambda x: len(x))
  label_distribution['num_mixed']=label_distribution['mixed'].apply(lambda x: len(x))
  dynasent  = pd.concat([data,label_distribution],axis=1)
  dynasent['disagreement'] = 1
  num_annotator = 5
  agreement_idx = [False if x==None else len(dynasent[x].iloc[idx])==num_annotator for idx,x in enumerate(dynasent['gold_label']) ]
  dynasent['disagreement'].loc[agreement_idx] = 0
  dynasent['disagreement_rate'] = [1-2/num_annotator if x==None else 1-len(dynasent[x].iloc[idx])/num_annotator for idx,x in enumerate(dynasent['gold_label']) ]
  dynasent['normalized_disagreement_rate'] = [dynasent['disagreement_rate'].iloc[x] / (1 - 2/num_annotator) for x in range(len(dynasent))]
  return dynasent

In [ ]:
train_dynasent = read_dynasent_json(r1_train)
dev_dynasent = read_dynasent_json(r1_dev)
test_dynasent = read_dynasent_json(r1_test)

In [ ]:
dynasent_frames = [train_dynasent, dev_dynasent, test_dynasent]
dynasent_disagreement = pd.concat(dynasent_frames)

In [ ]:
dynasent_disagreement.to_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/dynasent_disagreement.csv',index=False)

#Politeness
impolite: 1-8; neutral: 9-16; polite: 17-25. <br>
The neutral category is blurry range of impolite and polite. So it's hard to define a boundary of agree v.s. disagree. <br>
impolite: 1-13; polite 14-25. Only test impolite(0) and polite(1)


In [ ]:
!pip install convokit
from convokit import Corpus, download
corpus = Corpus(filename=download("wikipedia-politeness-corpus"))

In [ ]:
utterances = corpus.get_utterances_dataframe()
utterances['annotation_distribution'] =[list(utterances['meta.Annotations'].iloc[x].values()) for x in range(len(utterances))]
utterances['annotation_mean'] = [mean(x) for x in utterances['annotation_distribution']]

In [ ]:
all_annotation = list(set([a for b in utterances.annotation_distribution.tolist() for a in b]))
print( np.quantile(all_annotation, 0), np.quantile(all_annotation, .25) , np.quantile(all_annotation, .50), np.quantile(all_annotation, .75), np.quantile(all_annotation, 1))

1 7.0 13.0 19.0 25


In [14]:
print( np.quantile(utterances['annotation_mean'], 0), np.quantile(utterances['annotation_mean'], .25) , np.quantile(utterances['annotation_mean'], .50), np.quantile(utterances['annotation_mean'], .75), np.quantile(utterances['annotation_mean'], 1))

1 12.0 14.0 16.0 24


In [ ]:
print(utterances[utterances['meta.Binary']== -1]['annotation_mean'].describe())
print(utterances[utterances['meta.Binary']== 0]['annotation_mean'].describe())
print(utterances[utterances['meta.Binary']== 1]['annotation_mean'].describe())

count    1089.000000
mean        9.145087
std         2.323766
min         1.000000
25%         8.000000
50%        10.000000
75%        11.000000
max        14.000000
Name: annotation_mean, dtype: float64
count    2175.000000
mean       13.922299
std         1.405383
min         9.000000
25%        13.000000
50%        14.000000
75%        15.000000
max        18.000000
Name: annotation_mean, dtype: float64
count    1089.000000
mean       17.735537
std         1.517616
min        14.000000
25%        17.000000
50%        18.000000
75%        19.000000
max        24.000000
Name: annotation_mean, dtype: float64


In [ ]:
utterances['meta.Binary'].value_counts()

 0    2175
-1    1089
 1    1089
Name: meta.Binary, dtype: int64

In [ ]:
utterances_binary = utterances[(utterances['meta.Binary'] == -1) | (utterances['meta.Binary']  == 1)]
utterances_binary['meta.Binary'].value_counts()

-1    1089
 1    1089
Name: meta.Binary, dtype: int64

In [ ]:
utterances_binary['disagreement_rate'] = [ sum(i > 13 for i in x)/5 if mean(x)<=13 else sum(i <= 13 for i in x)/5 for x in utterances_binary['annotation_distribution']]

In [ ]:
#utterances_binary['disagreement_rate'].value_counts()
utterances_binary[utterances_binary['disagreement_rate']==0.6]

,timestamp,text,speaker,reply_to,conversation_id,meta.Normalized Score,meta.Binary,meta.Annotations,meta.parsed,vectors,annotation_distribution,annotation_mean,disagreement_rate
id,,,,,,,,,,,,,
377951,NOT_RECORDED,"Looks like you've been a bit inactive lately, ...",user,None,377951,-0.46268,-1,"{'A233ONYNWKDIYF': 17, 'A2UFD1I8ZO1V4G': 17, '...","[{'rt': 0, 'toks': [{'tok': 'Looks', 'tag': 'V...",[],"[17, 17, 17, 5, 1]",11,0.6
20054,NOT_RECORDED,Thanks for reverting that obnoxious vandalism ...,user,None,20054,-0.564319,-1,"{'A1T5YMBH35PDBX': 5, 'AO3XB5I5QNNUI': 17, 'A3...","[{'rt': 0, 'toks': [{'tok': 'Thanks', 'tag': '...",[],"[5, 17, 19, 16, 1]",11,0.6
390972,NOT_RECORDED,See my comment on the talk page. How did this ...,user,None,390972,-0.449209,-1,"{'A233ONYNWKDIYF': 13, 'A2UFD1I8ZO1V4G': 13, '...","[{'rt': 0, 'toks': [{'tok': 'See', 'tag': 'VB'...",[],"[13, 13, 17, 19, 9]",14,0.6
409120,NOT_RECORDED,"You're welcome to revert edits, but I'm puzzle...",user,None,409120,-0.467686,-1,"{'A2ZABRSFZKUOKV': 21, 'A3L8MJ1C4OEDGV': 19, '...","[{'rt': 20, 'toks': [{'tok': 'You', 'tag': 'PR...",[],"[21, 19, 14, 8, 1]",12,0.6


In [ ]:
utterances_binary[utterances_binary['conversation_id'] == '377951']['text'].iloc[0]

"Looks like you've been a bit inactive lately, but on the off-chance you check in soon, take a look at <url> and <url>.  You're probably the most familiar with CoM's style and brand of hostility, do you think Freakshownerd here is his sock?"

**didn't normalize the disagreement rate because it is different from categorical case. It has a different controversial case. The binary label is -1 but there are 3 out 5 think it is polite. However the two people who think it is -1 gave extreme low numbers.**

In [ ]:
utterances_binary.to_csv('/content/drive/MyDrive/Morality   Legality/Disagreement_Datasets/utterances_binary_disagreement.csv',index=False)

#Attitude